# LangChain `DocumentLoaders`

[LangChain](https://python.langchain.com/docs/get_started/introduction.html) is library that provides a kitchen sink of tools for LLMs, particularly integrating LLMs with other tools.  

One underrated feature of Langchain is [DocumentLoaders](https://integrations.langchain.com/), which allow you to acquire text data from any source, which is super useful even if you aren't using LLMs at all!  (It can also be useful to hijack these loaders to acquire data for fine tuning!)  

For example, if you are trying to get data from a website as text here are some useful `DocumentLoaders`:

1. [RecursiveURLLoader](https://python.langchain.com/docs/modules/data_connection/document_loaders/integrations/recursive_url_loader)
2. [SeleniumLoader](https://api.python.langchain.com/en/latest/document_loaders/langchain.document_loaders.url_selenium.SeleniumURLLoader.html)
3. [SitemapLoader](https://python.langchain.com/docs/modules/data_connection/document_loaders/integrations/sitemap): **this is explored below**.

I think it is useful to combine LangChain `DocumentLoaders` with [HuggingFace datasets](https://huggingface.co/docs/datasets/index), because it allows you to save, version and do other fun things like perform [semantic search of your data](https://huggingface.co/learn/nlp-course/chapter5/6?fw=tf) with FAISS.

As of this writing, there are [over 125 different kinds](https://integrations.langchain.com/) of `DocumentLoaders`.  I haven't been able to find a loader that isn't there to quickly acquire data I need.

## Sitemap Loader

[Sitemaps](https://en.wikipedia.org/wiki/Site_map) are a nice way to see a listing of all pages on a site.  This is useful for acquiring all of the text from a large site that might contain many pages.  Below, I use the [SitemapLoader](https://python.langchain.com/docs/modules/data_connection/document_loaders/integrations/sitemap) to get all of the text from [https://quarto.org](https://quarto.org).

:::{.callout-warning}
There is currently [a bug](https://github.com/hwchase17/langchain/issues/6521) in langchain, so I had to install an old version right before [this commit](https://github.com/hwchase17/langchain/pull/6107) which broke the `SitemapLoader`.  I had to downgrade to `v0.0.202` via `pip install langchain==0.0.202`
:::

In [ ]:
import nest_asyncio
nest_asyncio.apply() # you don't need this line outside notebooks
from langchain.document_loaders.sitemap import SitemapLoader
sitemap_loader = SitemapLoader(web_path="https://quarto.org/sitemap.xml")
sitemap_loader.requests_per_second = 4
docs = sitemap_loader.load()

Fetching pages: 100%|####################################| 269/269 [00:16<00:00, 16.21it/s]


In [ ]:
print(f'There are {len(docs)} pages')

There are 269 pages


Let's look at the content of one page:

In [ ]:
example = docs[0]
example.dict()

{'page_content': '\n\n\n\n\nQuarto - About Quarto\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nOverview\n\n\n\nGet Started\n\n\n\nGuide\n\n\n\nExtensions\n\n\n\nReference\n\n\n\nGallery\n\n\n\nBlog\n\n\n\nHelp\n\n\n\n\n\nReport a Bug\n\n\n\n\nAsk a Question\n\n\n\n\nFAQ\n\n\n\n\n \n\n\n\n\n\n \n\n\n\n\n\n\n\n\nOn this page\n\nGoals\nProject\nContribute\n\nEdit this pageReport an issue\n\n\n\n\n\nAbout Quarto\nOpen source tools for scientific and technical publishing\n\n\n\n\n\nGoals\nThe overarching goal of Quarto is to make the process of creating and collaborating on scientific and technical documents dramatically better. We hope to do this in several dimensions:\n\nCreate a writing and publishing environment with great integrated tools for technical content. We want to make authoring with embedded code, equations, figures, complex diagrams, interactive widgets, citations, cross references, and the myriad other special req

## Clean the data

When we look at this page, we can see a bunch of unwanted text.  The navbar and the sidenav are showing up, and we do not want this.   We can update the parsing function to fix this:

In [ ]:
from bs4 import BeautifulSoup


def remove_nav_and_header_elements(content: BeautifulSoup) -> str:
    exclude = content.find_all(["nav", "footer", "header", "head"])
    for element in exclude:
        element.decompose()

    return str(content.get_text()).strip()

In [ ]:
sitemap_loader = SitemapLoader(web_path="https://quarto.org/sitemap.xml",
                              parsing_function=remove_nav_and_header_elements)
sitemap_loader.requests_per_second = 4
docs = sitemap_loader.load()

Fetching pages: 100%|####################################| 269/269 [00:05<00:00, 52.00it/s]


In [ ]:
example = docs[0]
example.dict()

{'page_content': 'Goals\nThe overarching goal of Quarto is to make the process of creating and collaborating on scientific and technical documents dramatically better. We hope to do this in several dimensions:\n\nCreate a writing and publishing environment with great integrated tools for technical content. We want to make authoring with embedded code, equations, figures, complex diagrams, interactive widgets, citations, cross references, and the myriad other special requirements of scientific discourse straightforward and productive for everyone.\nHelp authors take full advantage of the web as a connected, interactive platform for communications, while still providing the ability to create excellent printed output from the same document source. Researchers shouldn’t need to choose between LaTeX, MS Word, and HTML but rather be able to author documents that target all of them at the same time.\nMake reproducible research and publications the norm rather than the exception. Reproducibili

# Create a HF Dataset

We can use the `from_list` method to load that sitemap data into a HF Dataset.

In [ ]:
from datasets import Dataset 
repo_name = 'hamel/quarto'

In [ ]:
quarto_data = Dataset.from_list([d.dict() for d in docs])

In [ ]:
quarto_data

Dataset({
    features: ['page_content', 'metadata'],
    num_rows: 269
})

In [ ]:
quarto_data.push_to_hub(repo_name)

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Updating downloaded metadata with the new split.


In [ ]:
#|echo: false
from IPython.display import Markdown
Markdown(f'This data is available at [https://huggingface.co/datasets/{repo_name}]({repo_name})')

This data is available at [https://huggingface.co/datasets/hamel/quarto](hamel/quarto)

## Download the data

You can download the data from the HuggingFace Hub like this:

In [ ]:
from datasets import load_dataset
remote_data = load_dataset(repo_name)

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Using custom data configuration hamel--quarto-b88699e31e28f953


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset parquet downloaded and prepared to /Users/hamel/.cache/huggingface/datasets/hamel___parquet/hamel--quarto-b88699e31e28f953/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
remote_data['train'][0]

{'page_content': 'Goals\nThe overarching goal of Quarto is to make the process of creating and collaborating on scientific and technical documents dramatically better. We hope to do this in several dimensions:\n\nCreate a writing and publishing environment with great integrated tools for technical content. We want to make authoring with embedded code, equations, figures, complex diagrams, interactive widgets, citations, cross references, and the myriad other special requirements of scientific discourse straightforward and productive for everyone.\nHelp authors take full advantage of the web as a connected, interactive platform for communications, while still providing the ability to create excellent printed output from the same document source. Researchers shouldn’t need to choose between LaTeX, MS Word, and HTML but rather be able to author documents that target all of them at the same time.\nMake reproducible research and publications the norm rather than the exception. Reproducibili

# GitHub Issues

We can use the [GitHubIssuesLoader](https://python.langchain.com/docs/modules/data_connection/document_loaders/integrations/github) to get all of the issues from a GitHub repo.

In [ ]:
from langchain.document_loaders import GitHubIssuesLoader

This assumes you have set the `GITHUB_PERSONAL_ACCESS_TOKEN` as an environment variable

In [ ]:
loader = GitHubIssuesLoader(
    repo="quarto-dev/quarto-cli",
    state='all', #get both open and closed issues
    include_prs=False,
)
quarto_issues = loader.load()

In [ ]:
len(quarto_issues)

2841

Wow, that's a lot of Issues!  Let's take a look at one:

In the issue below, I can see that it doesn't include comments.  We would have to get those separately with further API calls, but this is a good start!

In [ ]:
quarto_issues[0]

Document(page_content="### Bug description\n\nI am running Quarto to produce a report in which I mix python with R (using 95% R). When I started trying to use python, I believe that RStudio is not recognizing my python installation and packages:\r\n\r\n```\r\nError in py_call_impl(callable, call_args$unnamed, call_args$named) : \r\n  ModuleNotFoundError: No module named 'pandas'\r\n```\r\n\r\nAny ideas?\n\n### Steps to reproduce\n\n---\r\ntitle: 'Nota Técnica - Índice Socioeconômico dos Estudantes'\r\nauthor: ''\r\nformat: \r\n  html:\r\n    encoding: 'UTF-8'\r\n    theme: style.css\r\neditor: visual\r\nlang: pt\r\nexecute:\r\n  echo: false\r\n  warning: false\r\n---\r\n\r\n```{python}\r\n#| label: load-py-pckgs\r\nimport pandas as pd\r\nimport seaborn as sns\r\n```\n\n### Expected behavior\n\nI expected it to load python packages and modules with no problems.\n\n### Actual behavior\n\nError when loading any python package.\n\n### Your environment\n\nIDE: RStudio 2022.07.2 Build 576\r\

### Upload to the Hub

We can upload these issues to the hub like so, this will be available at [https://huggingface.co/datasets/hamel/quarto-issues](https://huggingface.co/datasets/hamel/quarto-issues)

In [ ]:
#|output: false
ds = Dataset.from_list([x.dict() for x in quarto_issues])
ds.push_to_hub('hamel/quarto-issues')

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
from datasets import load_dataset
remote_data = load_dataset('hamel/quarto-issues')

Downloading:   0%|          | 0.00/2.58k [00:00<?, ?B/s]

Using custom data configuration hamel--quarto-issues-52921768ee5c97fb


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset parquet downloaded and prepared to /Users/hamel/.cache/huggingface/datasets/hamel___parquet/hamel--quarto-issues-52921768ee5c97fb/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
remote_data['train'][0]

{'page_content': "### Bug description\n\nI am running Quarto to produce a report in which I mix python with R (using 95% R). When I started trying to use python, I believe that RStudio is not recognizing my python installation and packages:\r\n\r\n```\r\nError in py_call_impl(callable, call_args$unnamed, call_args$named) : \r\n  ModuleNotFoundError: No module named 'pandas'\r\n```\r\n\r\nAny ideas?\n\n### Steps to reproduce\n\n---\r\ntitle: 'Nota Técnica - Índice Socioeconômico dos Estudantes'\r\nauthor: ''\r\nformat: \r\n  html:\r\n    encoding: 'UTF-8'\r\n    theme: style.css\r\neditor: visual\r\nlang: pt\r\nexecute:\r\n  echo: false\r\n  warning: false\r\n---\r\n\r\n```{python}\r\n#| label: load-py-pckgs\r\nimport pandas as pd\r\nimport seaborn as sns\r\n```\n\n### Expected behavior\n\nI expected it to load python packages and modules with no problems.\n\n### Actual behavior\n\nError when loading any python package.\n\n### Your environment\n\nIDE: RStudio 2022.07.2 Build 576\r\nR ve